In [26]:
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import requests
import re
import pandas as pd
import numpy as np
import os

In [37]:
urls=["https://search.shopping.naver.com/catalog/10239983885?query=%EB%AF%B8%EB%B0%B4%EB%93%9C&NaPm=ct%3Dksek9d3s%7Cci%3Db3035c2e8e25f92d0e81c863f28e331614521b15%7Ctr%3Dslsl%7Csn%3D95694%7Chk%3D7ab2b79275e4376bcae34d7df9a0522a374c084f"]

In [38]:
for url in urls:
    
    shoppingmall_review = "/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[1]/ul/li[3]/a"

    shppingmall_review2 = "/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[2]/div/div[2]/ul/li[3]/a"
#                                                                v                                ^
    point5 = "/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/ul/li[2]/a"
    point4 = "/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/ul/li[3]/a"
    point3 = "/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/ul/li[4]/a"
    point2 = "/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/ul/li[5]/a"
    point1 = "/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/ul/li[6]/a"
    points_path = [point5, point4, point3, point2, point1]

    title_path = "/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li[1]/div[2]/div[1]/em"
    text_path = "/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li[1]/div[2]/div[1]/p"

    header = {'User-Agent': ''}
    d = webdriver.Chrome('chromedriver.exe') # webdriver = chrome
    d.implicitly_wait(3)
    d.get(url)
    req = requests.get(url)
    html = req.text
    soup = BeautifulSoup(html, "html.parser")

    a = d.find_element_by_xpath("/html/body/div/div/div[2]/div[2]/div[1]/h2")
    a = a.text #상품 제목


    elem = d.find_element_by_tag_name("body")
    elem.send_keys(Keys.PAGE_DOWN)
    sleep(2)

    d.find_element_by_xpath(shoppingmall_review).click()
    sleep(2)

    elem.send_keys(Keys.PAGE_DOWN)
    sleep(2)

    full_list = [[],[]]


    for i in range(len(points_path)):
        point = []
    
        d.find_element_by_xpath(shppingmall_review2).click()
        sleep(2)

    
        d.find_element_by_xpath(points_path[i]).click()
        print(points_path[i])
        sleep(2)
    
        page = 1
        starpoint = 5-i
    
        while True:
            html = d.page_source
            dom = BeautifulSoup(html, "lxml")
            titles_raw = dom.find_all("em", {"class":"reviewItems_title__39Z8H"})
            texts_raw = dom.find_all("p", {"class":"reviewItems_text__XIsTc"})


            try:
                d.find_element_by_xpath("/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[3]/a["+str(page)+"]").click()

                sleep(1)
                if page == 11:
                    page = 2


                html = d.page_source
                dom = BeautifulSoup(html, "lxml")
                titles_raw = dom.find_all("em", {"class":"reviewItems_title__39Z8H"})
                texts_raw = dom.find_all("p", {"class":"reviewItems_text__XIsTc"})
            
            
                for text_raw in texts_raw:
                    full_list[0].append(text_raw.text)
                    full_list[1].append(starpoint)
                    point.append(starpoint)

                if len(texts_raw) != 20:
                    break
        
                page = page + 1
            
                if len(point) > 400:
                    break
            
                sleep(4)
            except:
                for text_raw in texts_raw:
                    full_list[0].append(text_raw.text)
                    full_list[1].append(starpoint)
                    point.append(starpoint)
                break



    d.quit()
            
    df = pd.DataFrame(full_list)
    df = df.T
    df.columns = ["review", "star"]
    df.to_csv("C:/690009/TP2/final_project/member/NTJ/crawlerNSstar/data/"+a+".csv", encoding='utf-8-sig')


/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/ul/li[2]/a
/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/ul/li[3]/a
/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/ul/li[4]/a
/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/ul/li[5]/a
/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/ul/li[6]/a


In [39]:
df

,review,star
0,배터리는 오래가는 편이고 알람기능 하나 보고 샀어요.손목에서 진동이 울리니 너무 좋...,5
1,"대체로 만족합니다만, 액정 화면 밝기가 많이 약하네요 상품 만족합니다.배송 빠르기는...",5
2,미밴드 상위 버전들도 많지만 걸음 수 체크가 구매의 주 목적이었기 때문에 많은 기능...,5
3,막역하게 지내는 동생 생일선물로 사줬습니다.이 녀석 아주 샤오미라면 사족을 못 쓰는...,5
4,기존에 사용하던 제품 잃어버려서 재구매 했네요. 뭐 일단 가성비 좋은건 아시잖아요?...,5
...,...,...
1413,쓰기편한데 고무라서 그런지 얼마 안가 다시 사야할거같은 느낌적인느낌이들고 그래도 잘...,1
1414,상품평을 많은 기대를 갖고서 구입했으나 전화 수신시 알려만주고 번호는 나타나지 않으...,1
1415,금액대비 아주 좋습니다. 여러모로 편리하네요 .감사합니다.. 저렴하게 잘 산것 같아요..,1
1416,배송 일주일 걸림,1
